# Python и интернет. Модуль requests

**План**:

1. Requests
4. Beautiful Soup
5. Краулеры
6. API

## Как выкачать интернет
Современный Интернет предоставляет большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек: **requests** и **urllib**.

## Requests

Можно почитать в [документации](https://requests.readthedocs.io/en/latest/) или [тут](https://realpython.com/python-requests/).

Допустим, мы хотим скачать главную страницу Хабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [10]:
import requests

In [11]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит ответ сервера. Это не просто HTML-код страницы, а еще дополнительная информация. Если мы просто выведем этот `response`, нам покажется только код (если статус $-$ 200, то все ок; вам, вероятно, хорошо известен статус 404).

In [12]:
response

<Response [200]>

In [13]:
response.status_code

200

А вот в атрибуте `text` уже лежит HTML-код

In [14]:
print(response.text[:700])

<!DOCTYPE html>
<html lang="ru">

  <head>
    <title>Публикации &#x2F; Моя лента &#x2F; Хабр</title>
<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">
<link href="https://habr.com/ru/feed/" rel="canonical" data-hid="e3fa780">
<meta itemprop="image" content="/img/habr_ru.png">
<meta property="og:image" content="/img/habr_ru.png">
<meta property="og:image:width" content="1200">
<meta property="og:image:height" content="630">
<meta property="aiturec:image" content="/img/habr_ru.png">
<meta name="twitter:image" content="/img/habr_ru.png">
<meta property="vk:image" content="/img/habr_ru.png?format=vk">
<meta property="fb:app_id" content="444736788986613">
<meta property="fb:pages


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу в питоне, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5".
Но можно, например, представиться Мозиллой:

In [15]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get("https://habr.com/ru/", headers={'User-Agent': user_agent})

Или использовать специальную библиотеку

In [16]:
# !pip install fake_useragent --q

In [17]:
from fake_useragent import UserAgent

In [18]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Mobile Safari/537.36'

In [19]:
response = requests.get("https://habr.com/ru/", headers={'User-Agent': user_agent})

Можно посмотреть, что еще можно достать из response.

Функция ```dir``` выдает список методов и параметров объекта.

In [20]:
[i for i in dir(response) if not i.startswith("_")]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

- Кодировка:

In [21]:
response.encoding

'utf-8'

- Заголовки (техническая информация):

In [22]:
dict(response.headers)

{'Server': 'QRATOR',
 'Date': 'Thu, 06 Nov 2025 09:18:51 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Keep-Alive': 'timeout=15',
 'Vary': 'Accept-Encoding, Accept-Encoding',
 'X-DNS-Prefetch-Control': 'off',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Download-Options': 'noopen',
 'X-Content-Type-Options': 'nosniff',
 'X-XSS-Protection': '1; mode=block',
 'ETag': 'W/"5fc08-DbEsZ8b3l3ccBYs59LnNsccwkqs"',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains',
 'X-Request-Id': 'cf98f5340a1b192fd0d99ddf1193b8a8',
 'X-Request-Geoip-Country-Code': 'US',
 'X-Request-Detected-Device': 'mobile',
 'Content-Encoding': 'gzip'}

- Адрес запроса:

In [23]:
response.url

'https://habr.com/ru/feed/'

* Содержимое страницы:

In [24]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru">

  <head>
    <title>Публикации &#x2F; Моя лента &#x2F; Хабр</title>
<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">
<link href="https://habr.com/ru/feed/" rel="canonical" data-hid="e3fa780">
<meta itemprop="image" content="/img/habr_ru.png">
<meta 


Ура, всё на месте!

Но что всё это значит? Что такое HTML и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске HTML сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения ([плохой вариант](https://stackoverflow.com/questions/590747/using-regular-expressions-to-parse-html-why-not))
2. специальные библиотеки питона, например, BeautifulSoup или [lxml](https://lxml.de/) (хороший вариант)

## BeautifulSoup

[Документация](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [25]:
from bs4 import BeautifulSoup

Сначала инициализируем объект `BeautifulSoup`. Потом применим метод `find` и в скобочках укажем теги, по которым ищем. У некоторых тегов в HTML (как и в нашем случае) бывает еще и `class` и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем HTML файле.

In [26]:
soup = BeautifulSoup(response.text, 'html.parser')  # инициализируем (создаем) soup

post = soup.find('h2', {'class': 'tm-title tm-title_h2'})
print(post.get_text(), end="\n\n\n")
print(post.prettify())

Как мы автоматизировали код-ревью благодаря связке Aider + LLM


<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/fix_price/articles/963658/">
  <span>
   Как мы автоматизировали код-ревью благодаря связке Aider + LLM
  </span>
 </a>
 <!--]-->
</h2>



Но мы хотим получить все заголовки постов! Метод `find_all` возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [27]:
for post in soup.find_all('h2', {'class': 'tm-title tm-title_h2'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- ' * 10)  # для красоты

Как мы автоматизировали код-ревью благодаря связке Aider + LLM
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/fix_price/articles/963658/">
  <span>
   Как мы автоматизировали код-ревью благодаря связке Aider + LLM
  </span>
 </a>
 <!--]-->
</h2>

-- -- -- -- -- -- -- -- -- -- 
Часть ПК с Windows загружаются в режиме восстановления BitLocker после последних апдейтов
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/news/963634/">
  <span>
   Часть ПК с Windows загружаются в режиме восстановления BitLocker после последних апдейтов
  </span>
 </a>
 <!--]-->
</h2>

-- -- -- -- -- -- -- -- -- -- 
Java Digest #30
<h2 class="tm-title tm-title_h2" data-test-id="articleTitle">
 <!--[-->
 <a class="tm-title__link" data-article-link="tr

## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там проиcходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в HTML)

```
<article class="tm-articles-list__item" data-navigatable="" id="771476" tabindex="0">
 <div class="tm-article-snippet tm-article-snippet">
  <div class="tm-article-snippet__meta-container">
   <div class="tm-article-snippet__meta">
    <span class="tm-user-info tm-article-snippet__author">
     <a class="tm-user-info__userpic" href="/ru/users/darinka666/" title="darinka666">
      <div class="tm-entity-image">
       <img alt="" class="tm-entity-image__pic" height="24" src="https://assets.habr.com/habr-web/img/avatars/093.png" width="24"/>
      </div>
     </a>
     <span class="tm-user-info__user tm-user-info__user_appearance-default">
      <a class="tm-user-info__username" href="/ru/users/darinka666/">
       darinka666
       <!-- -->
      </a>
      <span class="tm-article-datetime-published">
       <time datetime="2023-11-02T09:22:25.000Z" title="2023-11-02, 12:22">
        11 минут назад
       </time>
      </span>
     </span>
    </span>
   </div>
   <!-- -->
  </div>
  <h2 class="tm-title tm-title_h2">
   <a class="tm-title__link" data-article-link="true" data-test-id="article-snippet-title-link" href="/ru/companies/mts_ai/articles/771476/">
    <span>
     Обзор Llemma: новая математическая open-source модель
    </span>
   </a>
  </h2>
  <div class="tm-article-snippet__stats">
   <div class="tm-article-complexity tm-article-complexity_complexity-medium">
    <span class="tm-svg-icon__wrapper tm-article-complexity__icon">
     <svg class="tm-svg-img tm-svg-icon" height="24" width="24">
      <title>
       Уровень сложности
      </title>
      <use xlink:href="/img/megazord-v28.2fb1b1c1..svg#complexity-medium">
      </use>
     </svg>
    </span>
    <span class="tm-article-complexity__label">
     Средний
    </span>
   </div>
```
(и так далее; часть вывода обрезана: например, нет закрывающего тега `</article>`)

## Задание на семинар 2

Скачать главную страницу Яндекс.Погоды и
    
- распечатать сегодняшнюю температуру и облачность
- распечатать время восхода и заката
- погоду на завтра

Простая версия: вместо Яндекс.Погоды возьмите [этот](https://simple-weather-website.netlify.app/) сайт и получите температуру, влажность и скорость ветра.

## Краулеры

## Что такое краулеры?

Краулеры $-$ это боты / программы, которые "ползают" (*crawl*) по страницам сайта и собирают информацию. Все чаще использование таких программ запрещается правилами пользования сайтами, поэтому это формально нехорошо. Но так продолжают делать и это надо уметь. Запрещают по 2 основным причинам: не хотят делиться данными и боятся, что вы уроните сервер (если сайт $-$ маленький, а сервер $-$ не очень, то это довольно легко).

Поэтому нужно собирать данные аккуратно, чтобы:
1. вас не заблокировали по IP,
2. вы не навредили серверу.

Вместо отдельных запросов для этого лучше создать сессию, которая позволит хранить информацию между запросами и поддерживать то же соединение и не создавать каждый раз все заново, что влияет на производительность.



In [55]:
from pprint import pprint # библиотека для красивого вывода

In [56]:
session = requests.session()

Попробуем сделать запрос, просто вместо requests.get мы пишет session.get.

In [57]:
response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/')

In [58]:
response

<Response [200]>

Посмотреть на `headers` запроса

In [61]:
pprint(dict(response.headers))

{'Cache-Control': 'no-cache',
 'Connection': 'keep-alive',
 'Content-Encoding': 'gzip',
 'Content-Type': 'text/html',
 'Date': 'Thu, 06 Nov 2025 10:36:36 GMT',
 'ETag': 'W/"690c7416-4a05c"',
 'Expires': 'Wed, 06 Nov 2024 10:36:36 GMT',
 'Keep-Alive': 'timeout=15',
 'Last-Modified': 'Thu, 06 Nov 2025 10:10:30 GMT',
 'Server': 'QRATOR',
 'Strict-Transport-Security': 'max-age=31536000;',
 'Transfer-Encoding': 'chunked',
 'Vary': 'Accept-Encoding',
 'X-Bitrix-Composite': 'Nginx (file)',
 'X-Content-Type-Options': 'nosniff',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-XSS-Protection': '1; mode=block',
 'n_cc': '0',
 'n_score': '0'}


### Стратегии сбора данных


По сути краулеры выполняют сбор страниц (их HTML) как мы это делали на прошлом занятии, но делают они это циклами (или циклами циклов). Можно выделить разные стратегии сбора данных:
    
**По типу навигации**

1. Все страницы со ссылками имеют удобные номера ("https://ficbook.net/fanfiction/no_fandom/originals?p=2"), обычно просто `p=<число>` или `page=<число>`. В этом случае вам нужно просто подставлять цифры подробнее про параметры передаваемые в ссылке можно посмотреть [здесь](https://en.wikipedia.org/wiki/Query_string).
2. Страницы называются как-то не структурированно (например, по названиям блоков). Тут нужно собирать ссылки на эти страницы и потом по ним ходить и собирать конечные странички.
3. Все расположено на одной страничке и догружается с использованием [WebSocket](https://en.wikipedia.org/wiki/WebSocket) или других технологий, при адрес в адресной строке никак не изменяется, данные могут догружаться на сайт автоматически по мере скролла страницы.

**По скорости обновления**

1. Если сайт довольно статичный по контенту (медленно появляются и удаляются материалы), то можно сначал собрать ссылки, а потом по ним ходить.
2. Если сайт очень динамичный по контенту (например, объявления на крупном сайте), вам нужно при получении страничкии ссылок сразу их обходить, а потом переходить к следующей, потому что ко времени получения исчерпывающего списка ссылок по сайту многие будут уже удалены или недоступны.



## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный IP, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:

1. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Google Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать `user-agent` в параметрах (а его выбирать случайно с помощью `fake_useragent`).
1. `time.sleep(x)` - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты).
2. `time.sleep(<случайный промежуток времени>)` - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль `random`).
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси) или использовать анонимизированные сети к примеру сеть [Tor](https://en.wikipedia.org/wiki/Tor_(network)) и аналоги.

При работе в Google Colab зачастую помогает перезапускать среду: код в Colab выполняется на внешних серверах, соответственно, при перезапуске будет отправлять запросы уже с другого сервера. Хотя иногда это приводит и к обратным проблемам: не все сайты доступны вне России, например (как и не все доступны из России))

### Притвориться нормальным браузером

In [ ]:
from fake_useragent import UserAgent

Можно настроить так, чтобы не проверять безопасность соединения, что иногда вызывает ошибки. Но это можно делать с сайтами, которым вы доверяете.

In [64]:
ua = UserAgent()

In [65]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/', headers=headers)

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3.1 Safari/605.1.15'}


### Пауза между запросами

In [66]:
import time
from datetime import datetime

In [68]:
for _ in range(5):
    response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/')
    print(datetime.now(), response)
    time.sleep(1)

2025-11-06 10:44:48.213423 <Response [200]>
2025-11-06 10:44:49.346097 <Response [200]>
2025-11-06 10:44:50.477801 <Response [200]>
2025-11-06 10:44:51.614423 <Response [200]>
2025-11-06 10:44:52.759634 <Response [200]>


Некоторые сайты замечают подозрительно регулярные запросы. `random.uniform` позволяет получить случайное число из отрезка и сделать наши запросы менее подозрительными.

In [69]:
import random

In [70]:
random.uniform(1, 3)

2.921613956926431

In [73]:
response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/')

In [76]:
for _ in range(5):
    response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/')
    print(datetime.now(), response)
    time.sleep(random.uniform(1.1, 5.2))

2025-11-06 10:49:38.084933 <Response [403]>
2025-11-06 10:49:40.527414 <Response [403]>
2025-11-06 10:49:44.607138 <Response [403]>
2025-11-06 10:49:46.532074 <Response [403]>
2025-11-06 10:49:50.334023 <Response [403]>


А всё, поздно уже.. (Статус 403 - Forbidden: сервер получил наш запрос, но отказался на него отвечать)

В идеале все перечисленные способы нужно использовать сразу, особенно если вы работаете локально, иначе сайт мжет вас запомнить и не пускать, например, несколько часов.

### Подключение через прокси

Прокси-сервер — это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

1. Адреса прокси можно взять со специальных сайтов: например, [https://hideip.me/ru/proxy/httplist](https://hideip.me/ru/proxy/httplist).
2. Потом проверить, что они рабочие, прежде чем использовать: [https://checkerproxy.net/](https://checkerproxy.net/).

In [78]:
proxy = {"https://": "https://176.213.141.107:8080"}

response = session.get('https://eksmo.ru/khudozhestvennaya-literatura/fantastika/', proxies=proxy)

## Пример

Давайте обкачаем немного новостей с сайта вышки.

1. Страницы имеют вид "https://www.hse.ru/news/page1.html", поэтому можно просто идти циклом.
2. Достанем дату публикации, заголовок, краткое описание (из станицы со списком новостей), текст полной статьи и метки (из самой страницы новости)
3. Сохраним в датафрейм.

In [79]:
from bs4 import BeautifulSoup
import re

### **Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу:

In [124]:
page_number = 3
url = f'https://www.hse.ru/news/page{page_number}.html'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

Распарсим с помощью `BeautifulSoup`:

In [125]:
soup = BeautifulSoup(page, 'html.parser')

Найдем отдельные посты:

In [126]:
news = soup.find_all('div', {'class': 'post'})

In [127]:
len(news)

10

Найдем заголовок-ссылку и запомним текст заголовка:

In [128]:
title_obj = news[0].find('a')
title_obj

<a class="link link_dark2 no-visited" href="/news/science/1097453616.html">«Развернуть обсуждение политики в области высшего образования в доказательное русло»</a>

In [129]:
title = title_obj.text
title

'«Развернуть обсуждение политики в области высшего образования в доказательное русло»'

Достанем свойства этой ссылки (куда ведет, `class`):

In [130]:
attrs = title_obj.attrs
attrs

{'href': '/news/science/1097453616.html',
 'class': ['link', 'link_dark2', 'no-visited']}

Достанем саму ссылку:

In [131]:
href = title_obj.attrs['href']
href

'/news/science/1097453616.html'

Достанем текст новости:

In [132]:
short_text = news[0].find('div', {'class': 'post__text'}).text
short_text

'29 октября в НИУ ВШЭ открылась XVI Международная конференция исследователей высшего образования (ИВО) на тему «Высшее образование: между частным и общественным благом». Для участия в конференции зарегистрировались более 600 человек из 32 регионов России и семи зарубежных стран, поступило рекордное число заявок на выступления с докладами — 242, из которых было принято 88.'

Достанем день, месяц, год публикации:

In [133]:
pub_day = news[0].find('div', {'class': 'post-meta__day'}).text
pub_day

'29'

In [134]:
pub_month = news[0].find('div', {'class': 'post-meta__month'}).text
pub_month

'окт'

In [135]:
pub_year = news[0].find('div', {'class': 'post-meta__year'}).text
pub_year

'2025'

### **Шаг 2. Научиться парсить страничку самой новости**

Скачаем и распарсим полученную ссылку:

In [139]:
url = 'https://www.hse.ru/' + href

req = session.get(url, headers={'User-Agent': ua.random})
page = req.text

soup = BeautifulSoup(page, 'html.parser')

Сохраним текст, распечатаем кусочек:

In [141]:
full_text = soup.find('div', {'class': 'post__content'}).text
full_text[:200]

'«Развернуть обсуждение политики в области высшего образования в доказательное русло»29 октября в НИУ ВШЭ открылась XVI Международная конференция исследователей высшего образования (ИВО) на тему «Высше'

Найдем теги, которые присвоены статье:

In [143]:
meta = soup.find('div', {'class': 'articleMeta'})

tags = meta.find_all('a', {'class': 'tag'})
tags = [t.text for t in tags]
tags

['исследования и аналитика', 'репортаж о событии']

### **Шаг 3. Оформляем нормально в функции**

Сделаем словарь соответствий имени месяца и его номера:

In [107]:
months = {
    value: key + 1
    for key, value in enumerate(
        ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    )
}

Парсим информацию из страницы со списком новостей (блок одной новости):

In [108]:
def parse_news_page_block(one_block):
    block = {}
    a = one_block.find('a')
    block['title'] = a.text
    block['href'] = a.attrs['href']
    block['short_text'] = one_block.find('div', {'class': 'post__text'}).text
    block['pub_day'] = int(one_block.find('div', {'class': 'post-meta__day'}).text)
    block['pub_month'] = months[one_block.find('div', {'class': 'post-meta__month'}).text]
    block['pub_year'] = int(one_block.find('div', {'class': 'post-meta__year'}).text)
    return block

Парсим отдельную страницу новости:

In [144]:
def parse_one_article(block):
    url_one = 'http://www.hse.ru' + block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find('div', {'class': 'post__content'}).text
    meta = soup.find('div', {'class': 'articleMeta'})
    tags = meta.find_all('a', {'class': 'tag'})
    block['tags'] = [t.text for t in tags]
    return block

Регулярное выражение для того, чтобы достать ID новости и не повторяться:

In [110]:
regex_hse_id = re.compile('/([0-9]*?).html')

Обработать N-ую страницу новостей:

In [145]:
def get_nth_page(page_number):
    # скачиваем
    url = f'https://www.hse.ru/news/page{page_number}.html'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    # находим новости
    news = soup.find_all('div', {'class': 'post'})

    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)

    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        if b['href'].startswith('/'):
            idx = regex_hse_id.findall(b['href'])[0]
            try:
                res = parse_one_article(b)
                res['hse_id'] = idx
                result.append(res)
            except Exception as e:
                print(e)

    # возвращаем найденную информацию
    return result

### **Шаг 4. Сохраняем в датафрейм**


In [146]:
from tqdm.auto import tqdm
import pandas as pd

Напишем функцию, куда передаем количество страниц и она выполняет все нужные действия:

In [147]:
def run_all(n_pages):
    blocks = []
    for i in tqdm(range(n_pages)):
        blocks.extend(get_nth_page(i+1))

    return blocks

Запускаем на 20 первых страниц:

In [152]:
blocks = run_all(3)

df = pd.DataFrame(blocks)

  0%|          | 0/3 [00:00<?, ?it/s]

In [154]:
df.sample(3)

,title,href,short_text,pub_day,pub_month,pub_year,full_text,tags,hse_id
2,Когда вирус наступает на мину: найден древний ...,/news/science/1099260087.html,"Когда вирус попадает в клетку, он вмешивается ...",5,11,2025,Когда вирус наступает на мину: найден древний ...,"[публикации, исследования и аналитика, програм...",1099260087
19,Исчезнувший сигнал: как солнечная активность з...,/news/science/1097022005.html,Исследователи из НИУ ВШЭ и ИКИ РАН проанализир...,28,10,2025,Исчезнувший сигнал: как солнечная активность з...,"[публикации, исследования и аналитика]",1097022005
5,Магистрант Вышки — семикратный чемпион России ...,/news/life/1098197960.html,В октябре Михаил Симонов вновь подтвердил титу...,1,11,2025,Магистрант Вышки — семикратный чемпион России ...,"[достижения, студенты, спорт]",1098197960


Посмотрим на 10 самых популярных тегов

In [155]:
from collections import Counter

In [156]:
tags = [tag for tags in df["tags"] for tag in tags]

In [157]:
for title, counts in sorted(Counter(tags).items(), key=lambda x: -x[-1])[:10]:
    print(counts,"\t", title)

9 	 исследования и аналитика
7 	 репортаж о событии
5 	 публикации
4 	 международное сотрудничество
3 	 центры превосходства
2 	 новое в ВШЭ
2 	 дополнительное образование
2 	 достижения
2 	 студенты
2 	 дискуссии


Посмотрим, сколько публикаций по месяцам

In [158]:
df.groupby("pub_month")["hse_id"].count()

,hse_id
pub_month,
10,16
11,7


## Задание на семинар

Сделайте краулер фильмов аналогичный тому, что выше.

У вас есть список на [Letterboxd](https://letterboxd.com/) c перечислением всех фильмов из книжки *1001 Movies You Must See Before You Die*: https://letterboxd.com/peterstanley/list/1001-movies-you-must-see-before-you-die/detail/.

1. Все страницы выглядят как `https://letterboxd.com/peterstanley/list/1001-movies-you-must-see-before-you-die/detail/page/<номер>/`, поэтому снова можно идти циклом.
2. Достанем номер фильма, его название, год выпуска со страницы списка.
3. Достанем режиссера, слоган (если есть) и краткое описание с самой страницы фильма.
4. Сохраним в датафрейм.

## API

Иногда можно не изгаляться с HTML, а воспользоваться специальным инструментом: **API**.

API расшифровывается как Application Programming Interface. Это набор правил, которыми одна программа может общаться и взаимодействовать с другой, в том числе с сайтами. Например, "сделай запрос к такой-то странице в таком-то формате, чтобы получить JSON с данными о погоде и не парсить HTML руками".

Но сначала пример попроще. Это сайт, который просто возвращает случайные картинки с лисами (есть [агрегатор](https://publicapi.dev/) разных API, там много такого можно найти):

In [30]:
fox = requests.get("https://randomfox.ca/floof/")

Обратите внимание, что `.text` вернет нам *строку*:

In [31]:
fox.text

'{"image":"https:\\/\\/randomfox.ca\\/images\\/98.jpg","link":"https:\\/\\/randomfox.ca\\/?i=98"}'

Но у респонза есть еще метод `.json()`. В данном случае он сразу приведет к списку или словарю. В данном случае словарь, так как API возвращает словарь с несколькими ключами.



In [32]:
fox.json()

{'image': 'https://randomfox.ca/images/98.jpg',
 'link': 'https://randomfox.ca/?i=98'}

Более сложный, но и более интересный пример: https://newsapi.org/

 На главной нужно получить ключ. Многие API доступны только по ключам: например, API гитхаба так определяет, к каким репозиториям у вас есть доступ, а к каким нет; некоторые API ограничивают число запросов на одного пользователя; и т.д.

 **Ключ $-$ это личная информация**, как, например, пароль. Ключ нежелательно вставлять в код, потому что будет легко забыть его удалить перед тем, как вы где-то этот код опубликуете. Лучше положить в отдельный файл и его читать:

In [35]:
with open("news_api_key.txt", "r") as f:
  api_key = f.read()

А если вы работаете в колабе, то можно положить ключ в секреты колаба и сделать вот так:

In [36]:
from google.colab import userdata

api_key = userdata.get('NEWS_API_KEY')

Какие параметры можно указывать в запросе, смотрите [здесь](https://newsapi.org/docs/endpoints/everything).

In [47]:
news = requests.get(
    f"https://newsapi.org/v2/everything?q=Apple&from=2025-11-5&sortBy=popularity&apiKey={api_key}"
    ).json()

In [48]:
news

{'status': 'ok',
 'totalResults': 492,
 'articles': [{'source': {'id': None, 'name': 'Android Central'},
   'author': 'bradypsnyder@gmail.com (Brady Snyder) , Brady Snyder',
   'title': "Google and Epic's settlement proposal could finally end the multi-year Play Store dispute",
   'description': 'Google and Epic spent years in legal battles over Play Store rules and transaction fees, but the sides have just agreed on a proposal to resolve them for good.',
   'url': 'https://www.androidcentral.com/apps-software/google-play-store/google-and-epics-settlement-proposal-could-finally-end-the-multi-year-play-store-dispute',
   'urlToImage': 'https://cdn.mos.cms.futurecdn.net/PwTCf9vyKJxtkDf6YBFTF7-2560-80.jpg',
   'publishedAt': '2025-11-05T06:45:23Z',
   'content': "What you need to know\r\n<ul><li>Google and Epic are proposing a joint settlement that would resolve over five years of legal battles.</li><li>The two companies' submitted the joint filing Tuesday, Nov… [+3806 chars]"},
  {'sourc

In [49]:
news["totalResults"]

492

In [50]:
articles = news["articles"][:5]

In [51]:
articles

[{'source': {'id': None, 'name': 'Android Central'},
  'author': 'bradypsnyder@gmail.com (Brady Snyder) , Brady Snyder',
  'title': "Google and Epic's settlement proposal could finally end the multi-year Play Store dispute",
  'description': 'Google and Epic spent years in legal battles over Play Store rules and transaction fees, but the sides have just agreed on a proposal to resolve them for good.',
  'url': 'https://www.androidcentral.com/apps-software/google-play-store/google-and-epics-settlement-proposal-could-finally-end-the-multi-year-play-store-dispute',
  'urlToImage': 'https://cdn.mos.cms.futurecdn.net/PwTCf9vyKJxtkDf6YBFTF7-2560-80.jpg',
  'publishedAt': '2025-11-05T06:45:23Z',
  'content': "What you need to know\r\n<ul><li>Google and Epic are proposing a joint settlement that would resolve over five years of legal battles.</li><li>The two companies' submitted the joint filing Tuesday, Nov… [+3806 chars]"},
 {'source': {'id': None, 'name': 'MacRumors'},
  'author': 'Juli Clo

Другие API:

1. [OpenWeatherMap](https://openweathermap.org/api) $-$ предоставляет информацию о погоде, прогнозы и исторические данные по всему миру. Есть бесплатный лимит запросов, но все равно требуется ввести данные банковской карты.

2. [GitHub](https://developer.github.com/v3/) $-$ API для доступа к данным о репозиториях, пользователях, коммитах и т.д.

3. [Open Trivia API](https://opentdb.com/api_config.php) $-$ API для получения вопросов и ответов из базы данных триивиума (викторины).

In [52]:
requests.get("https://opentdb.com/api.php?amount=10").json()

{'response_code': 0,
 'results': [{'type': 'multiple',
   'difficulty': 'medium',
   'category': 'Entertainment: Music',
   'question': 'What year was Weezer&#039;s album &quot;Pinkerton&quot; released? ',
   'correct_answer': '1996',
   'incorrect_answers': ['1990', '2001', '1994']},
  {'type': 'multiple',
   'difficulty': 'medium',
   'category': 'Science: Computers',
   'question': 'All of the following programs are classified as raster graphics editors EXCEPT:',
   'correct_answer': 'Inkscape',
   'incorrect_answers': ['Paint.NET', 'GIMP', 'Adobe Photoshop']},
  {'type': 'multiple',
   'difficulty': 'easy',
   'category': 'Science: Computers',
   'question': 'What does GHz stand for?',
   'correct_answer': 'Gigahertz',
   'incorrect_answers': ['Gigahotz', 'Gigahetz', 'Gigahatz']},
  {'type': 'multiple',
   'difficulty': 'hard',
   'category': 'Entertainment: Video Games',
   'question': 'How many voice channels does the Nintendo Entertainment System support natively?',
   'correct_

А еще есть Drama Corpus, который делали в том числе при участии Школы Лингвистики: https://dracor.org/doc/api

In [53]:
requests.get("https://dracor.org/api/corpora/rus/play/gogol-revizor/cast").json()[:5]

[{'numOfScenes': 21,
  'gender': 'MALE',
  'betweenness': 0.1264602852635307,
  'degree': 26,
  'numOfWords': 4991,
  'closeness': 0.8760416666666667,
  'id': 'gorodnichij',
  'numOfSpeechActs': 172,
  'name': 'Городничий',
  'isGroup': False,
  'eigenvector': 0.24182361124623364,
  'weightedDegree': 84},
 {'numOfScenes': 8,
  'gender': 'MALE',
  'betweenness': 0.010129431082777935,
  'degree': 21,
  'numOfWords': 748,
  'closeness': 0.7377192982456141,
  'id': 'ammos_fedorovich_ljapkin_tjapkin',
  'numOfSpeechActs': 49,
  'name': 'Аммос Федорович',
  'isGroup': False,
  'eigenvector': 0.23046261804194926,
  'weightedDegree': 54},
 {'numOfScenes': 10,
  'gender': 'MALE',
  'betweenness': 0.010129431082777935,
  'degree': 21,
  'numOfWords': 737,
  'closeness': 0.7377192982456141,
  'id': 'artemij_filippovich_zemljanika',
  'numOfSpeechActs': 51,
  'name': 'Артемий Филиппович Земляника',
  'isGroup': False,
  'eigenvector': 0.23046261804194926,
  'weightedDegree': 63},
 {'numOfScenes': 

## *Решение задания на семинар 1

In [29]:
for post in soup.find_all("article"):
    try:  # чтобы исключить случаи, когда нет юзернейма / времени / заголовка
        user_name = post.find('a', {'class': 'tm-user-info__username'}).get_text()
        time_article = post.find('time').get_text()
        header_article = post.find('h2', {'class': 'tm-title tm-title_h2'}).get_text()
        print('Name:', user_name)
        print('Time:', time_article)
        print('Header:', header_article, end='\n\n')
    except AttributeError:
        # ошибка AttributeError возникает, когда от объекта типа None вызывают метод .get_text()
        pass

Name: ksrepin 
Time: 1 минуту назад
Header: Как мы автоматизировали код-ревью благодаря связке Aider + LLM

Name: maybe_elf 
Time: 1 минуту назад
Header: Часть ПК с Windows загружаются в режиме восстановления BitLocker после последних апдейтов

Name: roma00712 
Time: 6 минут назад
Header: Java Digest #30

Name: alyonayurchenko 
Time: 15 минут назад
Header: РосНОУ вошёл в Университетский консорциум исследователей больших данных

Name: Realife 
Time: 17 минут назад
Header: Реставрация, которая меня сломала: Почему убрать смех из Скуби-Ду сложнее, чем сделать ремастер Тома и Джерри в 2к

Name: tvaleev 
Time: 18 минут назад
Header: Открыт доступ к ОС Аврора Developer Preview 5.2.0 для раннего тестирования

Name: LKamrad 
Time: 21 минуту назад
Header: Перед вами первый «торговый автомат» по продаже крепкого алкоголя. Вы не поверите, но ему уже почти три века

Name: maybe_elf 
Time: 21 минуту назад
Header: Canon установила 32-мегапиксельный сенсор и добавила поддержку видео 7K в камеру EOS R